In [1]:
using LinearAlgebra, Symbolics, Latexify, CairoMakie, Graphs, SimpleWeightedGraphs, GraphMakie
import Printf.@printf
import GraphMakie.SquareGrid, GraphMakie.Shell

## Quantum Systems

Now we change the weights on the graph to be complex values $c$ such that $|c|^2$ is between 0 and 1.
So now the weights aren't the actual probability of moving from one state to another, the norm is.
Personally, this takes away a good bit of the intuition of what the weights really mean physically.
But an important consequence of this change is that it is possible to add two complex numbers, and get a new complex number with a smaller norm, modeling the kind of interference that happens in quantum systems.

A state is now a vector of complex vectors whos **norms sum to one**, for example:

$$
X = \begin{bmatrix}
    \frac{1}{\sqrt{3}} & \frac{2i}{\sqrt{15}} & \sqrt{\frac{2}{5}}
\end{bmatrix}^T
$$

And now instead of graphs with real number weights, they have complex number weights.
Instead of requiring a doubly stochastic matrix, it requires a **unitary** matrix

Consider the graph:

![](2023-01-31-08-19-18.png)

And its corresponding matrix:

$$
U = \begin{bmatrix}
    \frac{1}{\sqrt{2}} & \frac{1}{\sqrt{2}} & 0 \\
    \frac{-i}{\sqrt{2}} & \frac{i}{\sqrt{2}} & 0 \\
    0 & 0 & i
\end{bmatrix}
$$

In [2]:
# Build the state vector
Xq = [
    1/sqrt(3)
    2*im/sqrt(15)
    sqrt(2/5)
]

# Display
@latexdefine(Xq, fmt="%.3f")

L"$Xq = \left[
\begin{array}{c}
0.577+0.000\mathit{i} \\
0.5163977794943222\mathit{i} \\
0.632+0.000\mathit{i} \\
\end{array}
\right]$"

In [3]:
# Can't build a graph object b/c the Graphs.jl package doesn't support complex weights ;-;
Mq = [
    1/sqrt(2) 1/sqrt(2) 0
    -im/sqrt(2) im/sqrt(2) 0
    0 0 im
]

# Display
@latexdefine(Mq, fmt="%.3f")

L"$Mq = \left[
\begin{array}{ccc}
0.707+0.000\mathit{i} & 0.707+0.000\mathit{i} & 0.0\mathit{i} \\
-0.7071067811865475\mathit{i} & 0.7071067811865475\mathit{i} & 0.0\mathit{i} \\
0.0\mathit{i} & 0.0\mathit{i} & \mathit{i} \\
\end{array}
\right]$"

In [12]:
# Check if the matrix is unitary
maybeunitary = Mq * Mq'

# Display
@latexdefine(maybeunitary, fmt="%.3f")

L"$maybeunitary = \left[
\begin{array}{ccc}
1.000+0.000\mathit{i} & 0.0\mathit{i} & 0.0\mathit{i} \\
0.0\mathit{i} & 1.000+0.000\mathit{i} & 0.0\mathit{i} \\
0.0\mathit{i} & 0.0\mathit{i} & 1.000+0.000\mathit{i} \\
\end{array}
\right]$"

We are also told that if you take the modulus squared of all the entries of a complex unitary matrix, you get a doubly stochastic matrix.

In [13]:
# Check if we get a doubly stochastic matrix
Mqnorm = norm.(Mq).^2

# Display
@latexdefine(Mqnorm, fmt="%.3f")

L"$Mqnorm = \left[
\begin{array}{ccc}
0.500 & 0.500 & 0.000 \\
0.500 & 0.500 & 0.000 \\
0.000 & 0.000 & 1.000 \\
\end{array}
\right]$"

Now we can see how the unitary matrix acts on states

In [6]:
# Get the next state
Yq₁ = Mq * Xq

# Display
@latexdefine(Yq₁, fmt="%.3f")

L"$Yq_1 = \left[
\begin{array}{c}
0.408+0.365\mathit{i} \\
-0.365-0.408\mathit{i} \\
0.6324555320336759\mathit{i} \\
\end{array}
\right]$"

In [7]:
# Check that the moduli sum to one
sum(norm.(Yq₁).^2)

0.9999999999999999

Since $M$ is now unitary, multiplying by the transpose should move the system back one time click, perfectly reversing the system

In [8]:
# Check that multiplying by the adjoing moves us back to the start
Xqmaybe = Mq' * Yq₁

# Display
display(@latexdefine(Xqmaybe, fmt="%.3f"))
display(@latexdefine(Xq, fmt="%.3f"))

L"$Xqmaybe = \left[
\begin{array}{c}
0.577+0.000\mathit{i} \\
0.5163977794943221\mathit{i} \\
0.632+0.000\mathit{i} \\
\end{array}
\right]$"

L"$Xq = \left[
\begin{array}{c}
0.577+0.000\mathit{i} \\
0.5163977794943222\mathit{i} \\
0.632+0.000\mathit{i} \\
\end{array}
\right]$"

In [9]:
Xqmaybe ≈ Xq

true

This means that $M * M^\dagger$ should be $I$

In [10]:
Imaybe = Mq * Mq'

# Display
@latexdefine(Imaybe, fmt="%.3f")

L"$Imaybe = \left[
\begin{array}{ccc}
1.000+0.000\mathit{i} & 0.0\mathit{i} & 0.0\mathit{i} \\
0.0\mathit{i} & 1.000+0.000\mathit{i} & 0.0\mathit{i} \\
0.0\mathit{i} & 0.0\mathit{i} & 1.000+0.000\mathit{i} \\
\end{array}
\right]$"

In [11]:
Imaybe ≈ I

true

### Ex: Quantum Billiard Ball

The revised graph for the system:

